In [4]:
import requests
import pandas as pd
from yelp_secret_key import api_key, api_key_secret

In [5]:
import json

# pull reviews

In [7]:
url = 'https://api.yelp.com/v3/businesses/search'
headers = {
    'Authorization': 'Bearer ' + api_key_secret,
}


In [7]:
url = 'https://api.yelp.com/v3/businesses/search'
headers = {
    'Authorization': 'Bearer ' + api_key_secret,
}


# Set up API parameters
params = {
    'categories': 'restaurants',
    'limit': 50
}

# Define zipcodes to search
zipcodes = ['92126', '92154', '92105', '92114', '92117', '92129', '92109', '92115', '92104', '92102']

# Count total number of restaurants
total_restaurants = 0

# Iterate over zipcodes and count restaurants in each
for zipcode in zipcodes:
    # Set location parameter to current zipcode
    params['location'] = zipcode
    # Make API call and parse response
    response = requests.get(url, headers=headers, params=params)
    data = json.loads(response.text)
    # Get total number of restaurants in current zipcode
    num_restaurants = data['total']
    # Add current count to total count
    total_restaurants += num_restaurants
    # Print current count
    print(f"{zipcode}: {num_restaurants}")
    
# Print total number of restaurants across all zipcodes
print(f"Total restaurants: {total_restaurants}")

92126: 302
92154: 2500
92105: 814
92114: 506
92117: 544
92129: 122
92109: 523
92115: 641
92104: 363
92102: 1500
Total restaurants: 7815


In [9]:
# Set up API parameters
params = {
    'categories': 'restaurants',
    'limit': 50
}

# Define zipcodes to search
zipcodes = ['92123', '92111', '92113', '92122', '92131', '92101', '92108', '92119', '92139', '92110']

# Count total number of restaurants
total_restaurants = 0

# Iterate over zipcodes and count restaurants in each
for zipcode in zipcodes:
    # Set location parameter to current zipcode
    params['location'] = zipcode
    # Make API call and parse response
    response = requests.get(url, headers=headers, params=params)
    data = json.loads(response.text)
    # Get total number of restaurants in current zipcode
    num_restaurants = data['total']
    # Add current count to total count
    total_restaurants += num_restaurants
    # Print current count
    print(f"{zipcode}: {num_restaurants}")
    
# Print total number of restaurants across all zip
print(f"Total restaurants: {total_restaurants}")

92123: 666
92111: 675
92113: 1300
92122: 1600
92131: 950
92101: 1600
92108: 990
92119: 347
92139: 101
92110: 958
Total restaurants: 9187


In [42]:
#luis revised list*********************
params = {
    'categories': 'restaurants',
    'limit': 50
}

zipcodes = ['92109','92104', '92110'] #(Pacific Beach),(North Park),#(USD/Linda Vista)

# Create empty dataframe to store results
df_sd_list1 = pd.DataFrame(columns=['id', 'name','category','rating', 'price','address', 'latitude', 'longitude'])

for zipcode in zipcodes:
    # Set location parameter to current zipcode
    params['location'] = zipcode
    offset = 0
    num_restaurants = 0
    while True:
        # Set offset parameter to retrieve next batch of restaurants
        params['offset'] = offset
        # Make API call and parse response
        response = requests.get(url, headers=headers, params=params)
        data = json.loads(response.text)
        # Get total number of restaurants in current zipcode
        num_restaurants = data['total']
        # Extract restaurant details from response and append to dataframe
        for business in data['businesses']:
            categories = [category['title'] for category in business['categories']]
            df_sd_list1 = df_sd_list1.append({
                'id': business['id'],
                'name': business['name'],
                'rating': business['rating'],
                'price': business.get('price', None),
                'category': categories,
                'address': ' '.join(business['location']['display_address']),
                'latitude': business['coordinates']['latitude'],
                'longitude': business['coordinates']['longitude']  
            }, ignore_index=True)
        # Check if there are any more results
        if num_restaurants <= offset + params['limit']:
            break
        # Increment offset for next batch
        offset += params['limit']
    # Print current count
    print(f"{zipcode}: {num_restaurants}")

# Print total number of restaurants across all zip
print(f"Total restaurants: {total_restaurants}")

92109: 523
92104: 363
92110: 958
Total restaurants: 3344


In [44]:
# Save the updated dataframe to a new CSV file
df_sd_list1.to_csv('sd_list1.csv', index=False)

df_sd_list1

,id,name,category,rating,price,address,latitude,longitude
0,3S6VtYgcXL8lRic0iV5GYQ,Sushi Ota,"[Sushi Bars, Japanese]",4.5,$$$,"4529 Mission Bay Dr San Diego, CA 92109",32.803664,-117.217125
1,PWdxcU5rjBIunreW8NKU2Q,Costa Brava,"[Tapas Bars, Spanish, Basque]",4.5,$$,"1653 Garnet Ave San Diego, CA 92109",32.799780,-117.239570
2,r-59c9YgjqShVvQVI-AGEQ,Pacific Beach Fish Shop,"[Seafood, Fish & Chips, Tacos]",4.0,$$,"1775 Garnet Ave San Diego, CA 92109",32.800362,-117.237298
3,_8y1B0HMhc3m9aLiVpp80w,Crushed,"[Tapas/Small Plates, Breakfast & Brunch, Bars]",4.0,$$,"967 Garnet Ave San Diego, CA 92109",32.797110,-117.252180
4,ssJJPFuV6tTDgHYe2w9csQ,Oscars Mexican Seafood,"[Mexican, Seafood]",4.5,$,"703 Turquoise St San Diego, CA 92109",32.808189,-117.260846
...,...,...,...,...,...,...,...,...
1839,P50ik8ozpsXmbUyIm2XHJA,Islandia Restaurant,[Restaurants],1.0,None,"1441 Quivira Rd San Diego, CA 92109",32.764230,-117.241250
1840,DKBKNpZ6s1zOpqd1B712lw,Tacos Memo,"[Food Trucks, Mexican]",5.0,None,"1500 3rd Ave Chula Vista, CA 91911",32.599670,-117.065815
1841,0s0VuNyb5FfcvD9CPkTing,Firebelly Wings,"[Food Delivery Services, Chicken Wings]",1.0,None,"San Diego, CA 92101",32.725080,-117.164330
1842,I26_9YN1UBHuRb01AFCVmw,The Wing Man,"[Food Delivery Services, Chicken Wings]",1.0,None,"San Diego, CA 92101",32.725080,-117.164330


In [ ]:
url = 'https://api.yelp.com/v3/businesses/search'
headers = {
    'Authorization': 'Bearer ' + api_key_secret,
}

In [50]:
# Load the restaurant data from a CSV file
df = pd.read_csv('sd_list1.csv')

# Load the reviews from the CSV file, if it exists
try:
    reviews_df = pd.read_csv('df_sd_list1_reviews.csv')
except FileNotFoundError:
    reviews_df = pd.DataFrame(columns=['id', 'reviews'])

# Merge the reviews dataframe with the restaurant dataframe
df = pd.merge(df, reviews_df, on='id', how='left')

# Set the maximum number of reviews to retrieve per restaurant
max_reviews_per_restaurant = 50

# Set the maximum number of API calls to make
max_api_calls = 4000

# Initialize the API call counter and the index of the last restaurant processed
api_calls = 0
last_index = reviews_df.index[-1] + 1 if not reviews_df.empty else 0

# Define a function to get reviews for a restaurant
def get_reviews(api_key, restaurant_id, offset, limit):
    url = f'https://api.yelp.com/v3/businesses/{restaurant_id}/reviews'
    headers = {'Authorization': f'Bearer {api_key_secret}'}
    params = {'offset': offset, 'limit': limit}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Loop over each restaurant in the dataframe, starting from the last processed index
for index, row in df.iloc[last_index:].iterrows():
    # Check if we have exceeded the maximum number of API calls
    if api_calls >= max_api_calls:
        break
    
    # Get the Yelp ID for the current restaurant
    restaurant_id = row['id']
    
    # Set up variables for pagination
    offset = 0
    limit = 3  # limit set to 3 to avoid hitting Yelp API rate limit
    
    # Loop until we have retrieved the maximum number of reviews for this restaurant
    reviews_list = []
    while len(reviews_list) < max_reviews_per_restaurant:
        # Get reviews for this restaurant
        api_key = api_key_secret
        response = get_reviews(api_key, restaurant_id, offset, limit)
        
        # Check if the API call was successful
        if response is None:
            break
        
        # Parse the reviews from the JSON response
        reviews = response.get('reviews', [])
        
        # Add the reviews to the list for this restaurant
        reviews_list.extend(reviews)
        
        # Update the offset for the next page of reviews
        offset += limit
        
        # Increment the API call counter
        api_calls += 1
        
        # Check if we have exceeded the maximum number of API calls
        if api_calls >= max_api_calls:
            break
    
    # Add the reviews as a new column to the dataframe
    df.loc[index, 'Reviews'] = str(reviews_list[:max_reviews_per_restaurant])
    
    # Save the dataframe to a CSV file after every restaurant to avoid losing progress
    df.to_csv('df_sd_list1_reviews.csv', index=False)
    
    # Clear the reviews list for the next restaurant
    reviews_list = []

In [55]:
df

,id,name,category,rating,price,address,latitude,longitude,reviews,Reviews
0,3S6VtYgcXL8lRic0iV5GYQ,Sushi Ota,"['Sushi Bars', 'Japanese']",4.5,$$$,"4529 Mission Bay Dr San Diego, CA 92109",32.803664,-117.217125,NaN,"[{'id': 'QNscmkFj08SGztzahb4Wkw', 'url': 'http..."
1,PWdxcU5rjBIunreW8NKU2Q,Costa Brava,"['Tapas Bars', 'Spanish', 'Basque']",4.5,$$,"1653 Garnet Ave San Diego, CA 92109",32.799780,-117.239570,NaN,"[{'id': '5uJi-60TLgf2LQCm0pcjiQ', 'url': 'http..."
2,r-59c9YgjqShVvQVI-AGEQ,Pacific Beach Fish Shop,"['Seafood', 'Fish & Chips', 'Tacos']",4.0,$$,"1775 Garnet Ave San Diego, CA 92109",32.800362,-117.237298,NaN,"[{'id': 'qKlLfo_jwxN3AXrlqfK3iQ', 'url': 'http..."
3,_8y1B0HMhc3m9aLiVpp80w,Crushed,"['Tapas/Small Plates', 'Breakfast & Brunch', '...",4.0,$$,"967 Garnet Ave San Diego, CA 92109",32.797110,-117.252180,NaN,"[{'id': 'lpJdSE3SN4APIMIW0t3IKQ', 'url': 'http..."
4,ssJJPFuV6tTDgHYe2w9csQ,Oscars Mexican Seafood,"['Mexican', 'Seafood']",4.5,$,"703 Turquoise St San Diego, CA 92109",32.808189,-117.260846,NaN,"[{'id': 'pQeeTPPXtG1LouMRR9J_LQ', 'url': 'http..."
...,...,...,...,...,...,...,...,...,...,...
1839,P50ik8ozpsXmbUyIm2XHJA,Islandia Restaurant,['Restaurants'],1.0,NaN,"1441 Quivira Rd San Diego, CA 92109",32.764230,-117.241250,NaN,NaN
1840,DKBKNpZ6s1zOpqd1B712lw,Tacos Memo,"['Food Trucks', 'Mexican']",5.0,NaN,"1500 3rd Ave Chula Vista, CA 91911",32.599670,-117.065815,NaN,NaN
1841,0s0VuNyb5FfcvD9CPkTing,Firebelly Wings,"['Food Delivery Services', 'Chicken Wings']",1.0,NaN,"San Diego, CA 92101",32.725080,-117.164330,NaN,NaN
1842,I26_9YN1UBHuRb01AFCVmw,The Wing Man,"['Food Delivery Services', 'Chicken Wings']",1.0,NaN,"San Diego, CA 92101",32.725080,-117.164330,NaN,NaN


In [54]:
df['Reviews'][0]

'[{\'id\': \'QNscmkFj08SGztzahb4Wkw\', \'url\': \'https://www.yelp.com/biz/sushi-ota-san-diego?adjust_creative=EAzmvVWISdT-BcqLuUJ2Ig&hrid=QNscmkFj08SGztzahb4Wkw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=EAzmvVWISdT-BcqLuUJ2Ig\', \'text\': \'Great customer service and amazing sushi! I needed a sushi platter and they let me order the rolls and they put them all on the platter and made it so...\', \'rating\': 5, \'time_created\': \'2023-02-10 11:18:03\', \'user\': {\'id\': \'-7ZrTFaosNcm-7GZZ-hKkQ\', \'profile_url\': \'https://www.yelp.com/user_details?userid=-7ZrTFaosNcm-7GZZ-hKkQ\', \'image_url\': None, \'name\': \'Quincy H.\'}}, {\'id\': \'X-puGGEvPLrd3qfpytlAdA\', \'url\': \'https://www.yelp.com/biz/sushi-ota-san-diego?adjust_creative=EAzmvVWISdT-BcqLuUJ2Ig&hrid=X-puGGEvPLrd3qfpytlAdA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=EAzmvVWISdT-BcqLuUJ2Ig\', \'text\': \'Sashimi Sampler, Spicy Salmon and Tempura Hand Rolls, Rainbow Ro